# Assignment 3 - Transliteration

### Setup [do not change]

In [ ]:
!git clone https://github.com/butoialexandra/eth-nlp-f22-hw3.git

import os
os.chdir('eth-nlp-f22-hw3')

!pip install -e .

Cloning into 'eth-nlp-f22-hw3'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 29 (delta 6), reused 16 (delta 2), pack-reused 0
Unpacking objects: 100% (29/29), 9.90 KiB | 921.00 KiB/s, done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/eth-nlp-f22-hw3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 2.7 MB/s eta 0:00:00
  Running setup.py develop for rayuela


In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1wlHbFChZFcBgEU8knAeaUIqE0W4EaJEC' -O 'test_cases.pkl'

--2023-02-06 20:07:51--  https://docs.google.com/uc?export=download&id=1wlHbFChZFcBgEU8knAeaUIqE0W4EaJEC
Resolving docs.google.com (docs.google.com)... 74.125.26.113, 74.125.26.101, 74.125.26.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.26.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-50-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/8493dab293a4ddilsrqh6j9s1cnojj7p/1675714050000/03249710210786382974/*/1wlHbFChZFcBgEU8knAeaUIqE0W4EaJEC?e=download&uuid=4bcda39b-9a1c-403b-b2c0-21183ba8f4d3 [following]
--2023-02-06 20:07:51--  https://doc-0s-50-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/8493dab293a4ddilsrqh6j9s1cnojj7p/1675714050000/03249710210786382974/*/1wlHbFChZFcBgEU8knAeaUIqE0W4EaJEC?e=download&uuid=4bcda39b-9a1c-403b-b2c0-21183ba8f4d3
Resolving doc-0s-50-docs.googleusercontent.com (doc-0s-50-docs.googleusercontent.com)... 172.217.203.132, 2607:f8b0:400

In [ ]:
from rayuela.base.semiring import Semiring, Real, Tropical
from rayuela.base.symbol import Sym, ε, ε_1, ε_2
from rayuela.base.misc import epsilon_filter

from rayuela.fsa.fst import FST
from rayuela.fsa.state import State, PairState

import dill
import numpy as np
from math import exp, log

from itertools import product

from frozendict import frozendict


### Subquestion a) - Log Semiring

In [ ]:
class Log(Semiring):
    def __init__(self, score):
        self.score = score

    def star(self):
        # TODO: implement Kleene star
        if self.score>=0:
            return Log(np.inf)
        else:
            return Log(np.log(1/(1-np.exp(self.score))))

    def __float__(self):
        return float(self.score)

    def __add__(self, other):
        # TODO: implement log addition
        # For numerical stability you should use the using the log-sum-exp trick
        # https://en.wikipedia.org/wiki/LogSumExp
        return Log(np.logaddexp(self.score, other.score))

    def __mul__(self, other):
        # TODO: implement log multiplication
        return Log(self.score + other.score)

    def __repr__(self):
        return f"{round(self.score, 15)}"

    def __eq__(self, other):
        return np.allclose(float(self.score), float(other.score), atol=1e-3)

    def __hash__(self):
        return hash(self.score)

# TODO: implement the log semiring one and zero
Log.zero = Log(-np.inf)
Log.one = Log(0)


Log.idempotent = False
Log.cancellative = True

### Before moving to the next subquestion, we give some examples of WFSTs to illustrate how to use the library `rayuela`.

In [ ]:
# A simple language model encoded by a WFST

sem = Real # the Real semiring
fst = FST(sem) # a WFST in the Real semiring

# add transitions
fst.add_arc(State(0), Sym('formal'), Sym('formal'), State(1), Real(0.2))
fst.add_arc(State(0), Sym('natural'), Sym('natural'), State(1), Real(0.3))
fst.add_arc(State(0), Sym('learning'), Sym('learning'), State(2), Real(0.2))
fst.add_arc(State(0), Sym('data'), Sym('data'), State(3), Real(0.3))

fst.add_arc(State(1), Sym('language'), Sym('language'), State(2), Real(0.6))
fst.add_arc(State(1), Sym('languages'), Sym('languages'), State(4), Real(0.4))

fst.add_arc(State(2), Sym('is'), Sym('is'), State(5), Real(1.0))

fst.add_arc(State(5), Sym('fun'), Sym('fun'), State(6), Real(1.0))

fst.add_arc(State(3), Sym('is'), Sym('is'), State(5), Real(0.5))
fst.add_arc(State(3), Sym('are'), Sym('are'), State(5), Real(0.5))

fst.add_arc(State(4), Sym('are'), Sym('are'), State(5), Real(1.0))

# set initial weights
# if not set, it defaults to semiring zero
fst.set_I(State(0), Real(1.0))
# set final weights
fst.set_F(State(6), Real(1.0))

# visualize
fst

WFSA(7 states, <class 'rayuela.base.semiring.Real'>)

In [ ]:
# A simple WFSA for transliteration

sem = Real # semiring
fst = FST(sem) # initialize WFST in the chosen semiring
one, zero = sem.one, sem.zero # semiring one and zero

# add transitions
fst.add_arc(State(0), Sym('d'), Sym('data'), State(1), Real(0.5))
fst.add_arc(State(0), Sym('d'), Sym('dew'), State(5), Real(0.5))

fst.add_arc(State(1), Sym('ey'), Sym('ÃŽÂµ'), State(2), Real(0.5))
fst.add_arc(State(1), Sym('ae'), Sym('ÃŽÂµ'), State(2), Real(0.5))

fst.add_arc(State(2), Sym('t'), Sym('ÃŽÂµ'), State(3), Real(0.7))
fst.add_arc(State(2), Sym('dx'), Sym('ÃŽÂµ'), State(3), Real(0.3))

fst.add_arc(State(3), Sym('ax'), Sym('ÃŽÂµ'), State(4), one)

fst.add_arc(State(5), Sym('uw'), Sym('ÃŽÂµ'), State(6), one)

# initial and final weights
fst.set_I(State(0), one)
fst.set_F(State(4), one)
fst.set_F(State(6), one)

fst

WFSA(7 states, <class 'rayuela.base.semiring.Real'>)

### Subquestion b) - String Transducer

In [ ]:
def string_fst(s, semiring=Log):
    """ Takes a string as input and a semiring
    and returns a transducer encoding of the
    string
    """
    fst = FST(semiring)

    # TODO: add arcs, initial and final weights
    # Arcs
    for i, char in enumerate(s):
        fst.set_arc(State(i), Sym(char), Sym(char), State(i+1), semiring(0.5))

    # Initial and final weights
    fst.set_I(State(0), semiring(0.5))
    fst.set_F(State(len(s)), semiring(0.5))

    return fst

In [ ]:
# you can visualise the transducer
fst = string_fst("abcd", semiring=Real)
fst

WFSA(5 states, <class 'rayuela.base.semiring.Real'>)

### Subquestion c) - Edit Distance Transducer

As we will not learn the weights on the arcs, you can set them to some pre-defined constants. You should choose the weights such that the pathsum that needs to be computed in e) does not diverge (this is caused by the weights of the self-loops in the transducer). 

In [ ]:
def edit_distance_fst(source_alphabet, target_alphabet, semiring=Log):
    """ Takes as input two sets of characters (source and target 
    alphabets) and a semiring and returns an edit-distance transducer
    """
    
    fst = FST(semiring)
    
    # TODO: add arcs, initial and final weights

    # Add arcs from each state to itself when output is epsilon (deletion):
    for target_char_in in source_alphabet:
            fst.set_arc(State(0), Sym(target_char_in), ε, State(0), semiring.one)

    for i, target_char in enumerate(target_alphabet, start=1): # state i for each char
        for source_char in source_alphabet:
            fst.set_arc(State(i), Sym(source_char), ε, State(i), semiring.one)

    # Add arcs from each state to all other states
    for j, target_char_target in enumerate(target_alphabet, start=1):
        for input_char in source_alphabet:
            fst.set_arc(State(0), Sym(input_char), Sym(target_char_target), State(j), semiring.one)
        fst.set_arc(State(0), ε, Sym(target_char_target), State(j), semiring.one)

    for i, target_char_source in enumerate(target_alphabet, start=1):
        for j, target_char_target in enumerate(target_alphabet, start=1):
            if i!=j:
                for input_char in source_alphabet:
                    fst.set_arc(State(i), Sym(input_char), Sym(target_char_target), State(j), semiring.one)
                fst.set_arc(State(i), ε, Sym(target_char_target), State(j), semiring.one)
    
    # Initial and final weights
    fst.set_I(State(0), semiring.one)
    fst.set_F(State(0), semiring.one)

    for i, _ in enumerate(target_alphabet, start=1):
        fst.set_F(State(i), semiring.one)
    
    return fst

In [ ]:
fst = edit_distance_fst({'a', 'b', 'c'}, {'d', 'e'}, semiring=Real)
fst

WFSA(3 states, <class 'rayuela.base.semiring.Real'>)

### Subquestion d) - Composition

We provide some composition test cases for you to inspect.

In [ ]:
with open('test_cases.pkl', 'rb') as in_file:
    examples = dill.load(in_file)

In [ ]:
def compose(fst1, fst2):
    """ Takes as input two transducers and returns
    their composition
    """
    # The machines must be in the same semiring
    assert fst1.R == fst2.R

    comp_fst = FST(R=fst1.R)

    # TODO: implement composition
    fst1_copy = fst1.copy()
    fst2_copy = fst2.copy()

    # Add initial state pairs to the stack and the visited set
    stack = []
    visited = set()
    for init_pair_1 in fst1_copy.I:
        for init_pair_2 in fst2_copy.I:
            stack.append((init_pair_1[0], init_pair_2[0], State(0)))
            visited.add((init_pair_1[0], init_pair_2[0], State(0)))

            new_init_state = PairState(init_pair_1[0], init_pair_2[0])
            init_weight = fst1_copy.λ[init_pair_1[0]]*fst2_copy.λ[init_pair_2[0]]
            comp_fst.set_I(new_init_state, init_weight)

    # Rename epsilon edges in T2: 
    for start_state in fst2_copy.δ.keys():
        temp = fst2_copy.δ[start_state].copy()
        for input_output in temp:
            if input_output[0] == ε:
                fst2_copy.δ[start_state][(ε_1, input_output[1])] = fst2_copy.δ[start_state][input_output]
                del fst2_copy.δ[start_state][input_output]

    # Rename epsilon edges in T1: 
    for start_state in fst1_copy.δ.keys():
        temp = fst1_copy.δ[start_state].copy()
        for input_output in temp:
            if input_output[1] == ε:
                fst1_copy.δ[start_state][(input_output[0], ε_2)] = fst1_copy.δ[start_state][input_output]
                del fst1_copy.δ[start_state][input_output]

    # Augment the epsilon transitions
    for state in fst1_copy.Q:
        fst1_copy.set_arc(state, ε, ε_1, state, fst1_copy.R.one)
    
    for state in fst2_copy.Q:
        fst2_copy.set_arc(state, ε_2, ε, state, fst2_copy.R.one)
    
    # Main loop
    while stack:
        q1, q2, qf = stack.pop()

        q1_transitions = [(ab, neighbour,w) for (ab, neighbour, w) in fst1_copy.arcs(q1)]
        q2_transitions = [(ab, neighbour,w) for (ab, neighbour,w) in fst2_copy.arcs(q2)]

        val_trans = []
        for arc1 in q1_transitions:
            a, b = arc1[0]
            start = arc1[1]
            w1 = arc1[2]
            for arc2 in q2_transitions:
                c, d = arc2[0]
                end = arc2[1]
                w2 = arc2[2]

                if epsilon_filter(b, c, qf) != State("⊥"):
                    val_trans.append(((start, (a,b), w1), (end, (c,d) ,w2)))

        for (start, (a,b), w1), (end, (c,d), w2) in val_trans:
            comp_fst.set_arc(PairState(q1,q2), a, d, PairState(start,end), w1*w2)
            
            qfp = epsilon_filter(b, c, qf)

            if (start,end,qfp) not in visited:
                stack.append((start,end,qfp))
                visited.add((start,end,qfp))

    # Set final states
    for pair in comp_fst.Q:
        wf = fst1_copy.ρ[pair.state1] * fst2_copy.ρ[pair.state2]
        if wf != fst1_copy.R.zero:
            comp_fst.add_F(pair, wf)

    return comp_fst

In [ ]:
example_id = 4

In [ ]:
# FST 1
examples[example_id]['fst1']['fst']

WFSA(2 states, <class 'rayuela.base.semiring.Real'>)

In [ ]:
# FST 2
examples[example_id]['fst2']['fst']

WFSA(2 states, <class 'rayuela.base.semiring.Real'>)

In [ ]:
# composition
examples[example_id]['composition']['fst']

WFSA(4 states, <class 'rayuela.base.semiring.Boolean'>)

In [ ]:
comp_fst = compose(examples[example_id]['fst1']['fst'], examples[example_id]['fst2']['fst'])
comp_fst

WFSA(4 states, <class 'rayuela.base.semiring.Boolean'>)

In [ ]:
# semiring
examples[example_id]['semiring']

rayuela.base.semiring.Real

### Subquestion e) - Lehmann's Algorithm

In [ ]:
class Pathsum:
    def __init__(self, fsa):

        # basic FSA stuff
        self.fsa = fsa
        self.R = fsa.R
        self.N = self.fsa.num_states

        # state dictionary
        self.I = {}
        for n, q in enumerate(self.fsa.Q):
            self.I[q] = n

        # lift into the semiring
        self.W, self.W_dict, self.alpha, self.beta = self.lift()

        
    def lift(self):
        """ creates the weight matrix, initial weight vector,
        final weight vector from the automaton """
        W = self.R.zeros((self.N, self.N))
        W_dict = {a: {} for a in self.fsa.Sigma}
        alpha = self.R.zeros((self.N,))
        beta = self.R.zeros((self.N,))
        for a in self.fsa.Sigma:
            for b in self.fsa.Delta:
                W_dict[a][b] = self.R.zeros((self.N, self.N))
        for p in self.fsa.Q:
            alpha[self.I[p]] += self.fsa.λ[p]
            beta[self.I[p]] += self.fsa.ρ[p]
            for (a, b), q, w in self.fsa.arcs(p):
                W[self.I[p], self.I[q]] += w
                W_dict[a][b][self.I[p], self.I[q]] = w
        return W, W_dict, alpha, beta
    
    def lehmann(self):
        """
        Lehmann's (1977) algorithm.
        """
        # TODO: implement
        
        # Initialize R:
        R = self.W

        # Main loop:
        for j in range(self.N):
            for i in range(self.N):
                for k in range(self.N):
                    R[i,k] = R[i,k] + R[i,j]*R[j,j].star()*R[j,k]

        # Add I to R:
        for i in range(self.N):
            R[i,i] = R[i,i] + self.R.one

        return R 

        # raise NotImplementedError

    def _iterate(self, K):
        P = self.R.diag(self.N)
        for n in range(K):
            P += self.W @ P
        return P

    def _fixpoint(self, K=200):
        if self.fsa.R.idempotent:
            return self._iterate(self.fsa.num_states)

        diag = self.R.zeros((self.fsa.num_states, self.fsa.num_states))
        for n in range(self.fsa.num_states):
            diag[n, n] = self.fsa.R.one
        P_old = diag

        for _ in range(K):
            P_new = diag + self.W @ P_old
            P_old = P_new

        return P_old

    def fixpoint(self):

        P = self._fixpoint()
        W = {}

        for p in self.fsa.Q:
            for q in self.fsa.Q:
                if p in self.I and q in self.I:
                    W[p, q] = P[self.I[p], self.I[q]]
                elif p == q:
                    W[p, q] = self.R.one
                else:
                    W[p, q] = self.R.zero

        return frozendict(W)

In [ ]:
pathsum = Pathsum(examples[3]['fst1']['fst'])
print(pathsum.fixpoint())
print(pathsum.lehmann())

frozendict.frozendict({(0, 0): 1.204687055617909, (0, 1): 0.049703748026656, (0, 2): 0.238528037012346, (1, 0): 0.423848011191222, (1, 1): 1.064607810824038, (1, 2): 0.083921906215862, (2, 0): 0.297575363071423, (2, 1): 0.220654517754702, (2, 2): 1.058919921888142})
[[1.210030479232752 0.052036796373853 0.249169852930657]
 [0.442313809320239 1.071795077141898 0.118150643517558]
 [0.315208576189901 0.226634327641494 1.085202126621377]]


### Subquestion f) - Normalizer

We provide some test cases for you to inspect.

In [ ]:
examples[0]['fst1']['Z'], examples[0]['fst2']['Z'], examples[0]['composition']['Z']

(0.00138425, 0.000997195294118, 7.07192371e-07)

In [ ]:
def normalizer(fst):
    """ Takes as input a finite-state transducer and
    returns its normalizer
    """
    
    pathsum = Pathsum(fst)
    # TODO: implement normalizer

    Z = fst.R.zero
    R = pathsum.lehmann()

    for i, qi in enumerate(fst.Q):
        for k, qk in enumerate(fst.Q):
            Z = Z + fst.λ[qi]*R[i,k]*fst.ρ[qk]

    return Z
    # raise NotImplementedError

In [ ]:
print((normalizer(examples[0]['fst1']['fst']),\
      normalizer(examples[0]['fst2']['fst']),\
      normalizer(examples[0]['composition']['fst'])))

(0.00138425, 0.000997195294118, 7.14910552e-07)


### Subquestion g) - Log-likelihood

In [ ]:
def log_likelihood(source_alphabet, target_alphabet, source_str, target_str):
    """ Takes as input the source and target alphabets (sets of characters)
    and a pair of source and target strings and returns the log-likelihood
    """

    # TODO: implement log-likelihood

    # Representing strings with WFSTs and edit distance WFST:
    source_fst = string_fst(source_str)
    target_fst = string_fst(target_str)
    edit_fst = edit_distance_fst(source_alphabet, target_alphabet)

    print('norm_source=',normalizer(source_fst))
    print('norm_target=',normalizer(target_fst))
    print('norm_edit=',normalizer(edit_fst))

    # Composing the WFSTs:
    TxTedit = compose(source_fst, edit_fst)
    TxTeditTy = compose(TxTedit, target_fst)

    # Computing the log likelihood as logp(y|x) = score(y,x) - log(Z)
    score = np.log(normalizer(TxTeditTy).score)
    logZ = np.log(normalizer(TxTedit).score)

    print('score=',score)
    print('logZ=', logZ)
     
    return score-logZ
    
    # raise NotImplementedError

### Subquestion h) - Decoding

In [ ]:
class Arctic(Semiring):
    def __init__(self, score):
        self.score = score

    def star(self):
        return self.one

    def __float__(self):
        return float(self.score)

    def __int__(self):
        return int(self.score)

    def __add__(self, other):
        return Arctic(max(self.score, other.score))

    def __mul__(self, other):
        if other is self.one:
            return self
        if self is self.one:
            return other
        if other is self.zero:
            return self.zero
        if self is self.zero:
            return self.zero
        return Arctic(self.score + other.score)

    def __invert__(self):
        return Arctic(-self.score)

    def __truediv__(self, other):
        return Arctic(self.score - other.score)

    def __lt__(self, other):
        return self.score < other.score

    def __repr__(self):
        return f"Arctic({self.score})"

    def __str__(self):
        return str(self.score)


Arctic.zero = Arctic(-float("inf"))
Arctic.one = Arctic(0.0)
Arctic.idempotent = True
Arctic.superior = True
Arctic.cancellative = True

In [ ]:
def to_semiring(fst, semiring):
    """ Lifts an WFST into a different semiring
    """
    oone, one = fst.R.one, semiring.one
    nfst = FST(semiring)

    # initial weights
    for q, w in fst.I:
        if w == oone:
            nfst.set_I(q, one)
        else:
            nfst.set_I(q, semiring(w.score))

    # final weights
    for q, w in fst.F:
        if w == oone:
            nfst.set_F(q, one)
        else:
            nfst.set_F(q, semiring(w.score))

    # arcs
    for p in fst.Q:
        for (a, b), q, w in fst.arcs(p):
            if w == oone:
                nfst.add_arc(p, a, b, q, one)
            else:
                nfst.add_arc(p, a, b, q, semiring(w.score))

    return nfst

In [ ]:
def decode(in_str, edit_distance_fst):
    """ Gets as input a source string and an
    edit-distance transducer and returns the weight of the most
    probable target string
    """
    # TODO: implement
    
    # Compose transducers
    str_fst = string_fst(in_str)
    comp_fst = compose(str_fst, edit_distance_fst)
    
    # Lift to arctic semiring
    comp_nfst = to_semiring(comp_fst, Arctic)

    # Run Lehmann -> Least probable string -> Arctic semiring
    output = normalizer(comp_nfst)
    
    return output
    # raise NotImplementedError

In [ ]:
fst = edit_distance_fst({'a', 'b', 'c'}, {'d', 'e'}, semiring=Log)
decode('aabcc', fst)

Arctic(3.5)